HOMEWORK 1
ES 2
PUNTO A

In [2]:
import networkx as nx

def find_perfect_matching(people_books_mapping):

    #---------- Creation of Grapf G ----------#

    G = nx.DiGraph()
    # Add nodes for people, books, source, and sink
    people_nodes = list(people_books_mapping.keys())
    book_nodes = set(book for books in people_books_mapping.values() for book in books)
    source_node = 'source'
    sink_node = 'sink'
    G.add_nodes_from(people_nodes, bipartite=0)
    G.add_nodes_from(book_nodes, bipartite=1)
    G.add_node(source_node)
    G.add_node(sink_node)

    # Connect source to people
    for person, books in people_books_mapping.items():
        G.add_edge(source_node, person, capacity=1)

    # Connect books to sink
    for book in book_nodes:
        G.add_edge(book, sink_node, capacity=1)

    # Connect people to books
    for person, books in people_books_mapping.items():
        for book in books:
            G.add_edge(person, book, capacity=1)
    #---------- End creation of Grapf G ----------#

    # Find the maximum flow
    _, flow_dict = nx.maximum_flow(G, source_node, sink_node, capacity='capacity')

    # Extract the matching from the flow
    perfect_matching = [(person, book) for person, books_flow in flow_dict.items() if person != source_node
                        for book, flow in books_flow.items() if flow == 1 and book != sink_node]

    return perfect_matching

# Example usage
people_books_mapping = {
    'p1': ['b1', 'b2'],
    'p2': ['b2', 'b3'],
    'p3': ['b1', 'b4'],
    'p4': ['b1', 'b2', 'b4']
}

perfect_matching = find_perfect_matching(people_books_mapping)
print("Perfect Matching:", perfect_matching)


Perfect Matching: [('p1', 'b2'), ('p2', 'b3'), ('p3', 'b1'), ('p4', 'b4')]


HOMEWORK 1
ES 2
PUNTO B


In [3]:
import networkx as nx

def assign_books(people_books_mapping, book_copies):

    #---------- Creation of Grapf G ----------#

    G = nx.DiGraph()
    # Add nodes for people, books, source, and sink
    people_nodes = list(people_books_mapping.keys())
    book_nodes = set(book for books in people_books_mapping.values() for book in books)
    source_node = 'source'
    sink_node = 'sink'
    G.add_nodes_from(people_nodes, bipartite=0)
    G.add_nodes_from(book_nodes, bipartite=1)
    G.add_node(source_node)
    G.add_node(sink_node)

    # Connect source to people
    for person, books in people_books_mapping.items():
        G.add_edge(source_node, person, capacity=float('inf'))

    # Connect books to sink with capacities based on available copies
    for book, copies in zip(book_nodes, book_copies):
        G.add_edge(book, sink_node, capacity=copies)

    # Connect people to books
    for person, books in people_books_mapping.items():
        for book in books:
            # Add an additional constraint for each edge
            G.add_edge(person, book, capacity=1)

    #---------- End creation of Grapf G ----------#

    # Find the maximum flow
    flow_value = nx.maximum_flow_value(G, source_node, sink_node)

    return flow_value

# Example usage with multiple copies of books (2; 3; 2; 2)
people_books_mapping = {
    'p1': ['b1', 'b2'],
    'p2': ['b2', 'b3'],
    'p3': ['b1', 'b4'],
    'p4': ['b1', 'b2', 'b4']
}

book_copies = [2, 3, 2, 2]

total_books_assigned = assign_books(people_books_mapping, book_copies)
print("Total Books Assigned:", total_books_assigned)


Total Books Assigned: 8


HOMEWORK 1
ES 2
PUNTO C

In [4]:
from functools import reduce
import networkx as nx

def sell_and_buy_books(people_books_mapping, book_copies):

     #---------- Creation of Grapf G ----------##

    G = nx.DiGraph()
    # Add nodes for people, books, source, and sink
    people_nodes = list(people_books_mapping.keys())
    book_nodes = sorted(list(set(book for books in people_books_mapping.values() for book in books)))
    source_node = 'source'
    sink_node = 'sink'
    G.add_nodes_from(people_nodes, bipartite=0)
    G.add_nodes_from(book_nodes, bipartite=1)
    G.add_node(source_node)
    G.add_node(sink_node)

    # Connect source to people with capacities
    for person, books in people_books_mapping.items():
        G.add_edge(source_node, person, capacity=float('inf'))

    # Connect books to sink with capacities based on available copies
    for book, copies in zip(book_nodes, book_copies):
        G.add_edge(book, sink_node, capacity=copies)

    # Connect people to books with capacities based on interests
    for person, books in people_books_mapping.items():
        for book in books:
            # Add an additional constraint for each edge
            G.add_edge(person, book, capacity=1)

    #---------- End creation of Grapf G ----------#

    # the in degree of book nodes tells how many people are interested in each book
    in_degrees = list(map(lambda node: G.in_degree(node), book_nodes))

    sold_books = []
    bougth_books = []
    for book, copies, possible_buyers in zip(book_nodes, book_copies, in_degrees):

        # (Example) If there are 2 copies of book3 and only 1 person interested in book3...
        if copies > possible_buyers:
            # ...then add (2-1=1) copy of book3 to the list of book to be sold
            sold_books.append((book,copies-possible_buyers))

        # If there are more people interested in a book than the number of copies available...
        elif copies < possible_buyers:
            # ...then that book needs to be bougth
            bougth_books.append((book,possible_buyers-copies))

    # Check if the number of sold books is consistent with the number of bougth books
    while (tot_sold:=reduce(lambda x,y: x+y, [b[1] for b in sold_books])) != (tot_bougth:=reduce(lambda x,y: x+y, [b[1] for b in bougth_books] )) :

        if tot_sold > tot_bougth:
            # decrement the number of sold copies of the last book in the list by one
            sold_books[-1] = (sold_books[-1][0],sold_books[-1][1]-1)
            if sold_books[-1][1]==0:
                # Remove the last book of the list if it has no copies to sell
                sold_books.pop()

        else:
            bougth_books[-1] = (bougth_books[-1][0],bougth_books[-1][1]-1)
            if bougth_books[-1][1]==0:
                bougth_books.pop()

    return sold_books, bougth_books

# Example usage with multiple copies of books (2; 3; 2; 2)
people_books_mapping = {
    'p1': ['b1', 'b2'],
    'p2': ['b2', 'b3'],
    'p3': ['b1', 'b4'],
    'p4': ['b1', 'b2', 'b4']
}

book_copies = [2, 3, 2, 2]

sold_books, bougth_books = sell_and_buy_books(people_books_mapping, book_copies)

print(f"(Sold_book,   qt_sold  ): {sold_books}")
print(f"(Bougth_book, qt_bougth): {bougth_books}")

(Sold_book,   qt_sold  ): [('b3', 1)]
(Bougth_book, qt_bougth): [('b1', 1)]


HOMEWORK 1
ES 2
PUNTO C, COMPARE RESULTS

In [5]:
import networkx as nx

# Example usage with multiple copies of books (2; 3; 2; 2)
people_books_mapping = {
    'p1': ['b1', 'b2'],
    'p2': ['b2', 'b3'],
    'p3': ['b1', 'b4'],
    'p4': ['b1', 'b2', 'b4']
}

book_copies = [2, 3, 2, 2]

# Sell and buy books to maximize the books assigned
sold_books, bougth_books = sell_and_buy_books(people_books_mapping, book_copies)

# Find the new book copies vector
book_nodes = sorted(list(set(book for books in people_books_mapping.values() for book in books)))
new_book_copies = []
# for each book find if it has been sold or bougth and decrease or increase its copies accordingly
for book, copies in zip(book_nodes, book_copies):
    if book in [b[0] for b in sold_books]:
        new_book_copies.append(copies - [qt for b,qt in sold_books if b==book ][0])
    elif book in [b[0] for b in bougth_books]:
        new_book_copies.append(copies + [qt for b,qt in bougth_books if b==book ][0])
    else :
        new_book_copies.append(copies)

# Compute the max assignment of books before and after the buy and selling
total_books_assigned     = assign_books(people_books_mapping, book_copies)
new_total_books_assigned = assign_books(people_books_mapping, new_book_copies)

# Compare results
print("Total Books Assigned with previous capacities of books:", total_books_assigned)
print("Total Books Assigned with new capacities of books:", new_total_books_assigned)
print(f"Sold books:   {sold_books}\nBougth books: {bougth_books}")


Total Books Assigned with previous capacities of books: 8
Total Books Assigned with new capacities of books: 7
Sold books:   [('b3', 1)]
Bougth books: [('b1', 1)]
